In [19]:
import re
import pandas as pd

from collections import Counter


In [20]:
import json


dataset = []
with open('../data/ViMedNER.txt', 'r', encoding='utf-8') as f:
    for dong in f:
        dataset.append(json.loads(dong.strip())) # .strip() để loại bỏ ký tự xuống dòng (\n) thừa

In [21]:
df = pd.DataFrame(dataset)
df.head()

,id,text,label,Comments
0,3504,kháng sinh dự phòng nếu đã được chỉ định trong...,"[[135, 163, DISEASE]]",[]
1,3505,ngày nay các chuyên gia tin rằng viêm nội tâm ...,"[[33, 49, DISEASE], [101, 109, CAUSE]]",[]
2,3506,hướng dẫn hiện nay khuyến cáo điều trị kháng s...,"[[129, 157, DISEASE]]",[]
3,3507,bác sĩ vẫn có thể khuyên nên dùng kháng sinh p...,"[[84, 100, TREATMENT]]",[]
4,3508,đối với hầu hết những người có thông liên thất...,"[[31, 46, DISEASE], [160, 176, DISEASE]]",[]


In [22]:
MAX_DISTANCE = 150

In [23]:
def calculate_distance(ent1, ent2):
    if ent1[1] < ent2[0]: return ent2[0] - ent1[1]
    elif ent2[1] < ent1[0]: return ent1[0] - ent2[1]
    return 0

In [24]:
import uuid
import json

# Giả định hàm calculate_distance và hằng số MAX_DISTANCE đã được định nghĩa ở trên
# MAX_DISTANCE = ...
# def calculate_distance(...): ...

ls_tasks = []

for row in dataset:
    text = row['text']
    origin_labels = row['label']
    
    task = {
        "data": {"text": text},
        "predictions": [{
            "model_version": "v1_rule_based",
            "result": []
        }]
    }
    
    entities_map = []
    
    # --- BƯỚC 1: Tạo Entities (Labels) ---
    for ent in origin_labels:
        start, end, label_name = ent
        
        region_id = str(uuid.uuid4())[:8]
        ner_result = {
            "id": region_id,
            "from_name": "label",
            "to_name": "text",
            "type": "labels",
            "value": {
                "start": start,
                "end": end,
                "text": text[start:end],
                "labels": [label_name]
            }
        }
        task["predictions"][0]["result"].append(ner_result)
        entities_map.append({
            "id": region_id,
            "start": start,
            "end": end,
            "label": label_name
        })

    # --- BƯỚC 2: Tạo Relations (Thực hiện sau khi đã có đủ hết entity của câu) ---
    # (Đưa đoạn này ra ngoài vòng lặp 'for ent in origin_labels')
    
    diseases = [e for e in entities_map if e['label'] == 'DISEASE']
    symptoms = [e for e in entities_map if e['label'] == 'SYMPTOM']
    treatments = [e for e in entities_map if e['label'] == 'TREATMENT']
    causes = [e for e in entities_map if e['label'] == 'CAUSE']
    diagnostics = [e for e in entities_map if e['label'] == 'DIAGNOSTIC']
    
    def add_relation(source, target, rel_name):
        dist = calculate_distance([source['start'], source['end']], 
                                  [target['start'], target['end']])
        if dist < MAX_DISTANCE:
            rel_result = {
                "from_id": source['id'],
                "to_id": target['id'],
                "type": "relation",
                "direction": "right",
                "labels": [rel_name]
            }
            task["predictions"][0]["result"].append(rel_result)
            
    # Rule: Disease -> Symptom
    for d in diseases:
        for s in symptoms:
            add_relation(d, s, "HAS_SYMPTOM")

    # Rule: Treatment -> Disease
    for t in treatments:
        for d in diseases:
            add_relation(t, d, "TREATS")

    # Rule: Cause -> Disease
    for c in causes:
        for d in diseases:
            add_relation(c, d, "CAUSES")
            
    # Rule: Diagnostic -> Disease
    for diag in diagnostics:
        for d in diseases:
            add_relation(diag, d, "DIAGNOSES")

    # --- BƯỚC 3: Append Task ---
    # (Quan trọng: Đặt ở đây, ngang hàng với 'task = ...', không thụt vào trong loop entity)
    ls_tasks.append(task)
        
with open("result_ls.json", 'w', encoding='utf-8') as f:
    json.dump(ls_tasks, f, ensure_ascii=False, indent=2)

In [25]:
ls_tasks

[{'data': {'text': 'kháng sinh dự phòng nếu đã được chỉ định trong quá khứ cần phải dùng kháng sinh trước khi thủ thuật nha khoa hoặc y tế để giảm nguy cơ viêm nội tâm mạc nhiễm khuẩn , nói chuyện với bác sĩ .'},
  'predictions': [{'model_version': 'v1_rule_based',
    'result': [{'id': '378fc1fc',
      'from_name': 'label',
      'to_name': 'text',
      'type': 'labels',
      'value': {'start': 135,
       'end': 163,
       'text': 'viêm nội tâm mạc nhiễm khuẩn',
       'labels': ['DISEASE']}}]}]},
 {'data': {'text': 'ngày nay các chuyên gia tin rằng viêm nội tâm mạc nhiều hơn với khả năng xảy ra từ việc tiếp xúc với vi trùng một cách ngẫu nhiên hơn từ một thủ thuật nha khoa hoặc phẫu thuật điển hình .'},
  'predictions': [{'model_version': 'v1_rule_based',
    'result': [{'id': 'bb7dfc0d',
      'from_name': 'label',
      'to_name': 'text',
      'type': 'labels',
      'value': {'start': 33,
       'end': 49,
       'text': 'viêm nội tâm mạc',
       'labels': ['DISEASE']}},
  

In [28]:
import json

def get_items_with_multiple_relations(data_list):
    """
    Trả về danh sách các item (dict) có số lượng relation > 1.
    """
    complex_items = [] # List chứa kết quả
    
    for item in data_list:
        predictions = item.get("predictions", [])
        
        if not predictions:
            continue
        
        # Lấy danh sách kết quả từ prediction đầu tiên
        results = predictions[0].get("result", [])
        
        # Đếm số relation
        num_relations = sum(1 for r in results if r.get("type") == "relation")
        
        # Nếu > 1 thì thêm toàn bộ item đó vào list
        if num_relations > 1:
            complex_items.append(item)
            
    return complex_items

filtered_list = get_items_with_multiple_relations(ls_tasks)

print(f"Tổng số câu tìm thấy: {len(filtered_list)}")


print("\n--- Ví dụ các câu tìm được ---")
for i, item in enumerate(filtered_list[:3]): # In 3 câu đầu
    text = item['data']['text']
    rels = sum(1 for r in item['predictions'][0]['result'] if r['type'] == 'relation')
    print(f"Câu {i+1} (Relations: {rels}): {text[:100]}...")

with open("complex_sentences_only.json", "w", encoding="utf-8") as f:
    json.dump(filtered_list, f, ensure_ascii=False, indent=2)
print("\nĐã lưu danh sách lọc ra file 'complex_sentences_only.json'")

Tổng số câu tìm thấy: 1543

--- Ví dụ các câu tìm được ---
Câu 1 (Relations: 9): bệnh viêm động mạch takayasu cũng có thể dẫn đến đau cánh tay hoặc đau ngực , huyết áp cao , và cuối...
Câu 2 (Relations: 2): suy tim do hở van động mạch chủ , viêm cơ tim hoặc động mạch chủ - tình trạng trong đó van động mạch...
Câu 3 (Relations: 2): đột quỵ , thiếu máu cục bộ , một loại đột quỵ xảy ra là kết quả của lưu lượng máu giảm hoặc bị tắc đ...

Đã lưu danh sách lọc ra file 'complex_sentences_only.json'


In [27]:
with open("result_ls_.json", 'w', encoding='utf-8') as f:
    json.dump(ls_tasks, f, ensure_ascii=False, indent=2)